In [1]:
#experiment name and snapshot folder (used for model persistence)
experiment_setup_name = "research.wikicat.sessions"
snapshot_path = "/home/jheuristic/yozhik/agentnet_snapshots/"

In [2]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline


#theano imports
#the problem is too simple to be run on GPU. Seriously.
%env THEANO_FLAGS='device=gpu2'
import theano
import theano.tensor as T
floatX = theano.config.floatX

import lasagne


env: THEANO_FLAGS='device=gpu2'


Using gpu device 2: Tesla K40m (CNMeM is disabled, CuDNN 4004)
/home/jheuristic/thenv/local/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [3]:
%load_ext autoreload
%autoreload 2

# This tutorial vuilds above the basic tutorial and shows several advanced tools
* multi-layer (and in principle, arbitrary) agent memory
* different reinforcement learning algorithms
* model persistence

__[todo: add more]__


# Experiment setup
* Here we load an experiment environment (description below)
* Designing one from scratch is explained in later tutorials

In [4]:
import agentnet.experiments.wikicat as experiment
print experiment.__doc__


This experiment Wikipedia data sample on musicians, scraped via the scripts present in this directory. 
For each musician, we know a number of boolean features (factors) on various topics like 
 * whether or not he/she was active in 1990's, 2000's, etc.
 * whether or not he/she plays guitar, piano, etc
 * whether or not he/she was born in 50's, 60's, etc.
 * what wikipedia categories does he/she belong to
 etc.



In the initial moment of time, agent knows nothing about any of them.
At each turn, agent may decide to
 - "open" one of the hidden factors.
   - if the factor turns out to be 1, agent receives +3 reward for Wikipedia categories, +1 for other categories,
   - Otherwise, the reward equals -1 for Wikipedia categories, -1 for other categores
   - all these rewards are parameterisable during environment creation
 - decide to quit session
   - yields reward of 0 and ends the interaction.
   - all farther actions will have no effect until next session

It is expected, that in orde

In [5]:
#Create an environment with all default parameters
env = experiment.WikicatEnvironment()

In [6]:
from sklearn.cross_validation import train_test_split
attrs, categories, feature_names = env.get_dataset()

train_attrs,test_attrs,train_cats,test_cats = train_test_split(attrs,categories,test_size=0.99,random_state=32)

print "train size:", train_attrs.shape,train_cats.shape
print "train size:", test_attrs.shape,test_cats.shape

print "features:",feature_names[::20]

train size: (85, 300) (85, 93)
train size: (8509, 300) (8509, 93)
features: ['Children:is_known', 'Instruments:vocals', 'Genres:electronic', 'Origin:scotland', 'Genres:jazz', 'Genres:electric', 'Occupation:bassist', 'Instruments:violin', 'Genres:instrumental', 'Occupation:entrepreneur', 'Instruments:mandolin', 'Instruments:vocalist', 'Genres:synthpop', 'decades_active:1990', 'Occupation:remixer', 'category:List_of_musical_artists_from_Japan', 'category:List_of_ambient_music_artists', 'category:List_of_crooners', 'category:List_of_country_music_performers', 'category:List_of_bass_guitarists']


In [7]:
env.load_random_batch(train_attrs,train_cats,5)

# agent setup
* An agent implementation has to contain three parts:
 * Memory layer(s)
  * in this case, we train two GRU layers [details below]
 * Q-values evaluation layers
  * in this case, a lasagne dense layer based on memory layer
 * Resolver - acton picker layer
  * in this case, the resolver has epsilon-greedy policy
  
  
### two-layer memory architecture
We train two memory states:
  * first one, based on observations,
  * second one, based on first one;

Note that here we update the second memory layer based on the CURRENT state
of the first one. Instead, you can try to feed it with a previous state.

The q-values are estimated on a concatenated state, effectively on both memory
states together, but there is no problem with limiting q-evaluator to only one:
just pass the correct gru layer as an incoming layer to the q-evaluator.

### Implementation:
We concatenate both memories into 1 state to pass it through the session loop.

To perform memory update, we need to slice the concatenated state back into
two memory states.

We do so by defining an input map function and passing it into agent.

We than concatenate two new states back to form a new memory state.



In [8]:
from agentnet.resolver import EpsilonGreedyResolver
from agentnet.memory import GRUMemoryLayer
from agentnet.agent import Agent

In [9]:
import lasagne
n_hid_1=512 #first GRU memory
n_hid_2=256 #second GRU memory
n_hid_3=256 #third GRU memory


_observation_layer = lasagne.layers.InputLayer([None,env.observation_size],name="obs_input")

_prev_gru1_layer = lasagne.layers.InputLayer([None,n_hid_1],name="prev_gru1_state_input")
_prev_gru2_layer = lasagne.layers.InputLayer([None,n_hid_2],name="prev_gru2_state_input")
_prev_gru3_layer = lasagne.layers.InputLayer([None,n_hid_3],name="prev_gru2_state_input")

#memory
gru1 = GRUMemoryLayer(n_hid_1,
                     _observation_layer,
                     _prev_gru1_layer,
                     name="gru1")

gru2 = GRUMemoryLayer(n_hid_2,
                     gru1,        #note that it takes CURRENT gru1 output as input.
                                  #replacing that with _prev_gru1_state would imply taking previous one.
                     _prev_gru2_layer,
                     name="gru2")

gru3 = GRUMemoryLayer(n_hid_3,
                     gru2,        #note that it takes CURRENT gru1 output as input.
                                  #replacing that with _prev_gru1_state would imply taking previous one.
                     _prev_gru3_layer,
                     name="gru2")


concatenated_memory = lasagne.layers.concat([gru1,gru2,gru3])

#q_eval
n_actions = len(feature_names)
q_eval = lasagne.layers.DenseLayer(concatenated_memory, #taking both memories. 
                                                        #Replacing with gru1 or gru2 would mean taking one
                                   num_units = n_actions,
                                   nonlinearity=lasagne.nonlinearities.linear,name="QEvaluator")
#resolver
epsilon = theano.shared(np.float32(0.9),"e-greedy.epsilon")

resolver = EpsilonGreedyResolver(q_eval,epsilon=epsilon,name="resolver")



#we need to define the new input map because concatenated_memory is a ConcatLayer and does not have default one

def custom_input_map(last_hidden,observation):
    """just a function that maps memory states to respective inputs"""
    return {
        _prev_gru1_layer:last_hidden[:,0:n_hid_1],
        _prev_gru2_layer:last_hidden[:,n_hid_1:n_hid_1+n_hid_2],
        _prev_gru3_layer:last_hidden[:,n_hid_1+n_hid_2:],
        _observation_layer:observation
    }

#all together
agent = Agent(concatenated_memory,q_eval,resolver,input_map=custom_input_map
             )


In [10]:
#Since it's a single lasagne network, one can get it's weights, output, etc
weights = lasagne.layers.get_all_params(resolver,trainable=True)
weights

[gru1.W_in_to_updategate,
 gru1.W_hid_to_updategate,
 gru1.b_updategate,
 gru1.W_in_to_resetgate,
 gru1.W_hid_to_resetgate,
 gru1.b_resetgate,
 gru1.W_in_to_hidden_update,
 gru1.W_hid_to_hidden_update,
 gru1.b_hidden_update,
 gru2.W_in_to_updategate,
 gru2.W_hid_to_updategate,
 gru2.b_updategate,
 gru2.W_in_to_resetgate,
 gru2.W_hid_to_resetgate,
 gru2.b_resetgate,
 gru2.W_in_to_hidden_update,
 gru2.W_hid_to_hidden_update,
 gru2.b_hidden_update,
 gru2.W_in_to_updategate,
 gru2.W_hid_to_updategate,
 gru2.b_updategate,
 gru2.W_in_to_resetgate,
 gru2.W_hid_to_resetgate,
 gru2.b_resetgate,
 gru2.W_in_to_hidden_update,
 gru2.W_hid_to_hidden_update,
 gru2.b_hidden_update,
 QEvaluator.W,
 QEvaluator.b]

## Agent setup in detail
* __Memory layers__
 * One-step recurrent layer
     * takes input and one's previous state
     * returns new memory state
   * Can be arbitrary lasagne layer
   * Several one-step recurrent units are implemented in __agentnet.memory__
   * Note that lasagne's default recurrent networks roll for several steps at once
     * in other words, __using lasagne recurrent units as memory means recurrence inside recurrence__
 * Using more than one memory layer is explained in farther tutorials


* __Q-values evaluation layer__
 * Can be arbitrary lasagne network
 * returns predicted Q-values for each action
 * Usually depends on memory as an input


* __Resolver__ - action picker
 * Decides on what action is taken
 * Normally takes Q-values as input
 * Currently all experiments require integer output
 * Several resolver layers are implemented in __agentnet.resolver__

# Interacting with environment
* an agent has a method that produces symbolic environment interaction sessions
* interactions result in sequences of observations, actions, q-values,etc
* one has to pre-define maximum session length.
 * in this case, environment implements an indicator of whether session has ended by current tick
* Since this environment also implements Objective methods, it can evaluate rewards for each [batch, time_tick]


In [11]:
#produce interaction sequences of length <= 10
history = agent.get_sessions(env,session_length=10,
                       batch_size=env.batch_size,)

state_seq,observation_seq,hidden_seq,qvalues_seq,action_seq = history

#get rewards for all actions
rewards_seq = env.get_reward_sequences(state_seq,action_seq)

#get indicator whether session is still active
is_alive_seq = env.get_whether_alive(observation_seq)


In [12]:

#Let us compile a function that returns all interaction logs
get_history = theano.function([],history+(is_alive_seq,rewards_seq,),mode=theano.compile.mode.FAST_RUN)



# [dev] Session pool init & update

In [13]:
from agentnet.environment import SessionPoolEnvironment,SessionBatchEnvironment

session_pool = SessionPoolEnvironment()

In [ ]:
#a new batch was loaded into MAIN environment
batch_size_interactive = 250
batch_size_replay = 100
pool_size = 1000
seq_length = 10


env.load_random_batch(train_attrs,train_cats,pool_size)

In [ ]:
#initialize the pool
initial_update_pool= session_pool.get_session_updates(observation_seq,action_seq,rewards_seq)


init_pool_fun = theano.function([],session_pool.pool_size,updates=initial_update_pool)
init_pool_fun()

In [ ]:
#add new sessions to pool

new_observation_seq = T.concatenate([session_pool.observations,observation_seq],axis=0)
new_action_seq = T.concatenate([session_pool.actions,action_seq],axis=0)
new_rewards_seq = T.concatenate([session_pool.rewards,rewards_seq],axis=0)


In [ ]:
#expand and shrink in one function


#temp environment that replayhs sessions of the expanded pool
expanded_pool = SessionBatchEnvironment(new_observation_seq,new_action_seq,new_rewards_seq)


replay_log = agent.get_sessions(expanded_pool,session_length=seq_length,
                       batch_size=expanded_pool.actions.shape[0],)


_,_,hidden_replay,qvalues_replay,untaken_actions_replay = replay_log
actions_replay = expanded_pool.actions
#in these sessions, states are fake single-zero placeholders, 
#observations are equal to session_pool.observations,
#untaken_actions_replay are actions that agent would want to take, but didn't
#session_pool.actions are the replayed actions


#Since actions did not change, we can use the old rewards
#TODO: add DontDoThatError to SessionEnvironment.get_reward_whatever
rewards_replay = expanded_pool.rewards

#indicator that i-th batch is alive by j-th timestamp
is_alive_replay = env.get_whether_alive(expanded_pool.observations)



In [ ]:
#reference replay Qvalues
from agentnet.learning import qlearning

#gamma - delayed reward coefficient - what fraction of reward is retained if it is obtained one tick later
gamma = theano.shared(np.float32(0.95),name = 'q_learning_gamma')

Qreference_replay = qlearning.get_reference(qvalues_replay,
                               actions_replay,
                               rewards_replay,
                               gamma_or_gammas=gamma,)



In [ ]:
#zero-out future rewards at session end
from agentnet.utils.mdp import get_end_indicator

is_end_replay = get_end_indicator(is_alive_replay).nonzero()

# "set reference Qvalues at end action ids to just the immediate rewards"
Qreference_replay = T.set_subtensor(Qreference_replay[is_end_replay],
                                    rewards_replay[is_end_replay])

In [ ]:
#prevent gradient updates over reference Qvalues (since they depend on predicted Qvalues)
from agentnet.utils import consider_constant
Qreference_replay = consider_constant(Qreference_replay)

In [ ]:
from agentnet.utils.mdp import get_action_Qvalues
action_Qvalues_replay = get_action_Qvalues(qvalues_replay,actions_replay)

In [ ]:
#Qvalues error

squared_error = lasagne.objectives.squared_error(action_Qvalues_replay,Qreference_replay)

#session mse = sum(errors_while_alive) / num_alive_ticks
replay_session_mse =  (squared_error * is_alive_replay).sum(axis=1) / T.sum(is_alive_replay,axis=1)



In [ ]:
order_by_error = T.argsort(replay_session_mse)

survived_ids = order_by_error[-pool_size:][::-1]

shrinked_observation_replay = expanded_pool.observations[survived_ids] 
shrinked_action_replay = expanded_pool.actions[survived_ids]
shrinked_rewards_replay = expanded_pool.rewards[survived_ids]


In [ ]:
#metrics
shrinked_pool_mse = replay_session_mse[survived_ids].mean()
avg_new_reward = T.sum(rewards_seq,axis=1).mean()
avg_pool_reward = T.sum(rewards_replay,axis=1).mean()

In [ ]:
from collections import OrderedDict

pool_updates = session_pool.get_session_updates(shrinked_observation_replay,shrinked_action_replay,shrinked_rewards_replay)

update_pool_fun = theano.function([],[shrinked_pool_mse,avg_new_reward,avg_pool_reward],updates=pool_updates)

# Replay training function
* training network on a small random batch of replays
 


In [ ]:
#get random sample of replay session pool [new sample each iteration]

session_batch = session_pool.sample_session_batch(max_n_samples=batch_size_replay,replace=True)


#get experience replay sessions
replay_batch = agent.get_sessions(session_batch,session_length=seq_length,
                       batch_size=session_batch.actions.shape[0],)


#just like when updating pool
_,_,hidden_batch,qvalues_batch,untaken_actions_batch = replay_batch
actions_batch = session_batch.actions


#Since actions did not change, we can use the old rewards
rewards_batch = session_batch.rewards

#indicator that i-th batch is alive by j-th timestamp
is_alive_batch = env.get_whether_alive(session_batch.observations)



In [ ]:
#get reference and action qvalues

Qreference_batch = qlearning.get_reference(qvalues_batch,
                               actions_batch,
                               rewards_batch,
                               gamma_or_gammas=gamma,)

is_end_batch = get_end_indicator(is_alive_batch).nonzero()

# "set reference Qvalues at end action ids to just the immediate rewards"
Qreference_batch = T.set_subtensor(Qreference_batch[is_end_batch],
                                    rewards_batch[is_end_batch])

In [ ]:
from agentnet.utils.mdp import get_action_Qvalues
action_Qvalues_batch = get_action_Qvalues(qvalues_batch,actions_batch)

#### Define loss functions

In [ ]:
#tensor of elementwise squared errors
batch_mse = lasagne.objectives.squared_error(Qreference_batch,action_Qvalues_batch)

#zero-out ticks after session ended
batch_mse *= is_alive_batch

#compute average of squared error sums per session
batch_mse_loss = batch_mse.sum() / is_alive_batch.sum()

In [ ]:
#regularize network weights

from lasagne.regularization import regularize_network_params, l2
reg_l2 = regularize_network_params(resolver,l2)*10**-4

In [ ]:
batch_loss = batch_mse_loss + reg_l2

#### Compute weight updates

In [ ]:

updates = lasagne.updates.adadelta(batch_loss,weights,learning_rate=0.05)

#### Some auxilary evaluation metrics

In [ ]:
mean_batch_reward = rewards_batch.sum(axis=1).mean()
#...

# Compile train and evaluation functions

In [ ]:
train_fun = theano.function([],[batch_loss,mean_batch_reward],updates=updates)

evaluation_fun = theano.function([],[batch_loss,batch_mse_loss,reg_l2,mean_batch_reward])

# session visualization tools


* this is a completely optional step of visualizing agent's sessions as chains of actions
* usually useful to get insight on what worked and what din't
* in this case, we print strings following pattern
  * [action_name] ([predicted action qvalue]) -> reward [reference qvalue] | next iteration

* plot shows
    * time ticks over X, abstract values over Y
    * bold lines are Qvalues for actions
    * dots on bold lines represent what actions were taken at each moment of time
    * dashed lines are agent's hidden state neurons
    * blue vertical line - session end
    
    
__Warning! the visualization tools are underdeveloped and only allow simple operations.__

if you found yourself struggling to make it do what you want for 5 minutes, go write your own tool [and contribute it :)]


In [ ]:
reference_Qvalues_seq = qlearning.get_reference(qvalues_seq,
                               action_seq,
                               rewards_seq,
                               gamma_or_gammas=gamma,)

is_end_seq = get_end_indicator(is_alive_seq).nonzero()

# "set reference Qvalues at end action ids to just the immediate rewards"
reference_Qvalues_seq = T.set_subtensor(reference_Qvalues_seq[is_end_seq],
                                    rewards_seq[is_end_seq])

In [ ]:
from agentnet.display.sessions import print_sessions
get_printables = theano.function([], [
        hidden_seq,qvalues_seq, action_seq,rewards_seq,reference_Qvalues_seq,is_alive_seq
    ])

def display_sessions(with_plots = False):
        
    hidden_log,qvalues_log,actions_log,reward_log, reference_qv_log, is_alive_log = get_printables()
    
    
    print_sessions(qvalues_log,actions_log,reward_log,
                   is_alive_seq = is_alive_log,
                   #hidden_seq=hidden_log, #do not plot hidden since there's too many actions already
                   reference_qvalues_seq = reference_qv_log,
                   action_names=feature_names,
                   legend = False, #do not show legend since there's too many labeled objects
                  plot_qvalues = with_plots)

In [ ]:
#visualize untrained network performance (which is mostly random)
env.load_random_batch(train_attrs,train_cats,1)
display_sessions(with_plots=True)

# Training loop

In [ ]:
#tools for model persistence
from agentnet.utils.persistence import save,load
import os

In [ ]:
from agentnet.display import Metrics
score_log = Metrics()

In [ ]:
#starting epoch
epoch_counter = 1

#moving average estimation
alpha = 0.1
ma_reward_current = -7.
ma_reward_greedy = -7.
ma_reward_pool = -7

In [ ]:
n_epochs = 1000000

for i in range(n_epochs):    
    
    #train
    loss,avg_reward = train_fun()
    
    
    #update pool
    if epoch_counter %10==0:

        
        #reward loss moving average
        env.load_random_batch(train_attrs,train_cats,batch_size_interactive)
        _,avg_reward,avg_pool_reward = update_pool_fun()
        ma_reward_pool = alpha*avg_pool_reward+ (1-alpha)*ma_reward_pool
        score_log["session pool e-greedy reward"][epoch_counter] = ma_reward_pool
    
    
    ##update resolver's epsilon (chance of random action instead of optimal one)
    if epoch_counter%10 ==0:
        current_epsilon =  0.05 + 0.95*np.exp(-epoch_counter/10000.)
        resolver.epsilon.set_value(np.float32(current_epsilon))
    
    
    
    ##record current learning progress and show learning curves
    if epoch_counter%100 ==0:

        ##update learning curves
        full_loss, q_loss, l2_penalty, avg_reward_current = evaluation_fun()
        ma_reward_current = (1-alpha)*ma_reward_current + alpha*avg_reward_current
        score_log["expected e-greedy reward"][epoch_counter] = ma_reward_current
        
        
        
        #greedy train
        resolver.epsilon.set_value(0)
        avg_reward_greedy = evaluation_fun()[-1]
        ma_reward_greedy = (1-alpha)*ma_reward_greedy + alpha*avg_reward_greedy
        score_log["expected greedy reward"][epoch_counter] = ma_reward_greedy
        
        
        #back to epsilon-greedy
        resolver.epsilon.set_value(np.float32(current_epsilon))

        print "epoch %i,loss %.5f, epsilon %.5f, rewards: ( e-greedy %.5f, greedy %.5f) "%(
            epoch_counter,full_loss,current_epsilon,ma_reward_current,ma_reward_greedy)
        print "rec %.3f reg %.3f"%(q_loss,l2_penalty)

    if epoch_counter %1000 ==0:
        print "Learning curves:"
        score_log.plot()


        print "Random session examples"
        env.load_random_batch(train_attrs,train_cats,3)
        display_sessions(with_plots=False)
    
    #save snapshot
    if epoch_counter %10000 ==0:
        snap_name = "{}.epoch{}.pcl".format(os.path.join(snapshot_path,experiment_setup_name), epoch_counter)
        save(resolver,snap_name)
        print "saved", snap_name

        
        
    epoch_counter  +=1

    
# Time to drink some coffee!

# Evaluating results

In [ ]:
score_log.plot("final")

In [ ]:
print "Random session examples"
env.load_random_batch(train_attrs,train_cats,10)
display_sessions(with_plots=True)


In [ ]:
#load earlier snapshot
snap_name = "{}.epoch{}.pcl".format(os.path.join(snapshot_path,experiment_setup_name), 10000)
load(resolver,snap_name)
        

In [ ]:
print "Earlier session examples"
env.load_random_batch(train_attrs,train_cats,3)
display_sessions(with_plots=True)
